In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [ff71e718] MixedModels v4.4.0 `~/.julia/dev/MixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

#### using HighDimMixedModels

In [3]:
using HighDimMixedModels
#using highDimMM

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [4]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [5]:
#f = @formula(period ~ 0 + herd + incid + hsz)
#f = @formula(period ~ 0 +(1|herd) + incid + hsz )
f = @formula(period ~ 0 + incid + hsz + (1|herd))
contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

Dict{Symbol, UnionAll} with 3 entries:
  :hsz   => ContinuousTerm
  :herd  => CategoricalTerm
  :incid => ContinuousTerm

In [6]:
f

FormulaTerm
Response:
  period(unknown)
Predictors:
  0
  incid(unknown)
  hsz(unknown)
  (herd)->1 | herd

**try for apply_schema**

In [7]:
LinearMixedModel<:MixedModel

true

In [8]:
highDimMixedModel<:MixedModel

true

In [ ]:
# try for f2
f2 = @formula(period ~ 0  +(1|herd) + incid + hsz)
#f2 = @formula(period ~ 0  + incid + hsz +(1|herd) )

In [ ]:
#sch = schema(f2,df,contrasts)
sch = schema(f2,df)

In [ ]:
form = apply_schema(f2, sch)

In [ ]:
form = apply_schema(f2, sch, LinearMixedModel)  ## do not need MixedModels prefix due to multiple dispatch

In [ ]:
form = apply_schema(f2, sch, highDimMixedModel)

In [ ]:
modelcols(form,df)

### Construct object

#### 4. parse random effect in formula

In [9]:
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [10]:
first(HMM.M.M,10)

10-element Vector{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0
 1.0
 1.0
 8.0
 2.0
 0.0

In [11]:
first(HMM.X.X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [12]:
size(HMM.Z.Z)

(56, 15)

In [13]:
sch = schema(df)
form = apply_schema(f, sch, highDimMixedModel)

FormulaTerm
Response:
  period(continuous)
Predictors:
  0
  incid(continuous)
  hsz(continuous)
  (1 | herd)

In [ ]:
# when the random effect is the last term, can use form directly instead of form.rhs
FixedEffectMatrix,Z = modelmatrix(form,df)

In [ ]:
modelmatrix(form.rhs,df)

In [ ]:
y, pre = modelcols(form,df)

In [ ]:
numOfHDM = 1
M = highDimMat(FixedEffectMatrix[:,1:numOfHDM])
X = XMat(FixedEffectMatrix[:, (numOfHDM + 1):size(FixedEffectMatrix,2)])

In [ ]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

In [ ]:
ml = MixedModels.fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

In [ ]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

In [ ]:
HMM::highDimMixedModel

In [ ]:
HMM::MixedModel

In [ ]:
HMM::StatsModels.RegressionModel

In [ ]:
ml::LinearMixedModel

In [ ]:
ml::MixedModel

In [ ]:
ml::StatsModels.RegressionModel

In [ ]:
HMM::MixedModel

In [ ]:
HMM::StatsModels.RegressionModel

In [ ]:
highDimMixedModel::MixedModel

In [ ]:
HMM

In [ ]:
HMM.formula

### Now try to use apply_schema in MixedModels

In [ ]:
# try for f2
f2 = @formula(period ~ 0 + incid + hsz +(1|herd))
sch = schema(f2,df)
form = apply_schema(f2, sch, highDimMixedModel) ## multiple dispatch

In [ ]:
y,X = modelcols(form,df)

In [ ]:
Z = X[2]

In [ ]:
X[1]

In [ ]:
df.herd

In [ ]:
form.rhs

### Optimization

In [14]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 16 	 Time: 0:00:00 (30.78 ms/it)


(Linear mixed model fit by REML
 period ~ 0 + incid + hsz + (1 | herd)
 REML criterion at convergence: 219.11119977822892

Variance components:
            Column   VarianceStd.Dev.
herd     (Intercept)  1.33834 1.15687
Residual              1.82132 1.34956
 Number of obs: 56; levels of grouping factors: 15

  Fixed-effects parameters:
─────────────────────────────────────────────
           Coef.  Std. Error      z  Pr(>|z|)
─────────────────────────────────────────────
incid  -0.293874   0.0890102  -3.30    0.0010
hsz     0.131742   0.0238714   5.52    <1e-07
─────────────────────────────────────────────, Initial parameter vector: [1.0]
Initial objective value:  219.28154799380482

Optimizer (from NLopt):   LN_BOBYQA
Lower bounds:             [0.0]
ftol_rel:                 1.0e-12
ftol_abs:                 1.0e-8
xtol_rel:                 0.0
xtol_abs:                 [1.0e-10]
initial_step:             [0.75]
maxfeval:                 -1
maxtime:                  -1.0

Function eva

In [19]:
display(ml)

(Linear mixed model fit by REML
 period ~ 0 + incid + hsz + (1 | herd)
 REML criterion at convergence: 219.11119977822892

Variance components:
            Column   VarianceStd.Dev.
herd     (Intercept)  1.33834 1.15687
Residual              1.82132 1.34956
 Number of obs: 56; levels of grouping factors: 15

  Fixed-effects parameters:
─────────────────────────────────────────────
           Coef.  Std. Error      z  Pr(>|z|)
─────────────────────────────────────────────
incid  -0.293874   0.0890102  -3.30    0.0010
hsz     0.131742   0.0238714   5.52    <1e-07
─────────────────────────────────────────────, Initial parameter vector: [1.0]
Initial objective value:  219.28154799380482

Optimizer (from NLopt):   LN_BOBYQA
Lower bounds:             [0.0]
ftol_rel:                 1.0e-12
ftol_abs:                 1.0e-8
xtol_rel:                 0.0
xtol_abs:                 [1.0e-10]
initial_step:             [0.75]
maxfeval:                 -1
maxtime:                  -1.0

Function eva

In [15]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = 57.1099018030012
The initial object value is 57.1099018030012
sigma = [1.0, 1.0]
negLog = 57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = 57.1099018030012
sigma = [1.75, 1.0]
negLog = 55.97912076973617
sigma = [1.0, 1.75]
negLog = 52.37720093978967
sigma = [0.25, 1.0]
negLog = 64.50473446075652
sigma = [1.0, 0.25]
negLog = 128.22481961167495
sigma = [1.510389333579731, 1.4284710320133605]
negLog = 52.78019245866251
sigma = [0.8451353314889427, 1.437431307110896]
negLog = 53.411902302579605
sigma = [1.0, 1.9]
negLog = 52.31808747690741
sigma = [0.822599313455345, 1.9607062304338283]
negLog = 52.490735469672636
sigma = [1.048013829974836, 1.957616595969803]
negLog = 52.30520784011165
sigma = [1.0036098308862278, 2.0180590054596546]
negLog = 52.357982155798666
sigma = [1.0739875442712838, 1.9305681918459645]
negLog = 52.28417164563711
sigma = [1.1350679280066387, 1.887045936358205]
negLog = 52.251667221635145
sigma = [1.22890885574

In [ ]:
opt

In [17]:
betaM

1-element Vector{Float64}:
 -0.2938738291611581

In [16]:
betaX

1-element Vector{Float64}:
 0.1317420744001128